# Installs

In [1]:
# %pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q


This may take a while

In [2]:
# !pip install torchsummaryX==1.3.0
# !pip install pandas==1.5.2
# !pip install wandb --quiet
# !pip install python-Levenshtein -q
# !git clone --recursive https://github.com/parlance/ctcdecode.git
# !pip install wget -q
# %cd ctcdecode
# !pip install . -q
# %cd ..

# Imports

In [3]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

/home/shiwon/miniconda3/envs/idl_hw3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device:  cuda


# Kaggle Setup

In [4]:
# !pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8 -q
# !mkdir /home/shiwon/.kaggle

# with open("/home/shiwon/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"shiwonkim","key":"8eeeb84558070cf64f4a51d1897ead6a"}')

# !chmod 600 /home/shiwon/.kaggle/kaggle.json

In [5]:
# !kaggle competitions download -c 11-785-hw3p2-f24

In [6]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f24-hw3p2  ctcdecode  hw3p2asr-f24.zip  sample_data
'''
# !unzip -q hw3p2-785-f24.zip
# !ls

'\nThis will take a couple minutes, but you should see at least the following:\n11-785-f24-hw3p2  ctcdecode  hw3p2asr-f24.zip  sample_data\n'

# Dataset and Dataloader

In [7]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [8]:
# You might want to play around with the mapping as a sanity check here

### Train Data

In [9]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    def __init__(self, root, partition="train-clean-100", phonemes=PHONEMES):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        self.transcript_dir = os.path.join(root, partition, "transcript")

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.PHONEMES = phonemes

        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        assert self.length == len(self.transcript_files)

        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
        self.mfccs, self.transcripts = [], []
        
        for i in range(self.length):
            mfcc = np.load(os.path.join(self.mfcc_dir, self.mfcc_files[i]))
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / (np.std(mfcc, axis=0) + 1e-8) # Cepstral normalization
            self.mfccs.append(mfcc)

            transcript = np.load(os.path.join(self.transcript_dir, self.transcript_files[i]))[1:-1] # Remove SOS and EOS
            self.transcripts.append([self.PHONEMES.index(p) for p in transcript])

    def __len__(self):

        '''
        What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        mfcc = torch.FloatTensor(self.mfccs[ind])
        transcript = torch.LongTensor(self.transcripts[ind])
        
        return mfcc, transcript

    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [item[0] for item in batch]
        # batch of output phonemes
        batch_transcript = [item[1] for item in batch]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)
        lengths_mfcc = [mfcc.shape[0] for mfcc in batch_mfcc]

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True)
        lengths_transcript = [transcript.shape[0] for transcript in batch_transcript]

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.LongTensor(lengths_mfcc), torch.LongTensor(lengths_transcript)



### Test Data

In [10]:
# Test Dataloader
#TODO

class AudioDatasetTest(torch.utils.data.Dataset):
    
    def __init__(self, root, partition="test-clean"):

        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.length = len(self.mfcc_files)
        
        self.mfccs = []

        for mfcc_name in self.mfcc_files:
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_name))
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / (np.std(mfcc, axis=0) + 1e-8)
            self.mfccs.append(mfcc)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        mfcc = torch.FloatTensor(self.mfccs[ind])

        return mfcc
    
    def collate_fn(self, batch):
        batch_mfcc_pad = pad_sequence(batch, batch_first=True)
        lengths_mfcc = [mfcc.shape[0] for mfcc in batch]

        return batch_mfcc_pad, torch.LongTensor(lengths_mfcc)

### Config - Hyperparameters

In [11]:
root = '/home/shiwon/2024_CMU/IntroDL/HW3/P2/11785-f24-hw3p2'

# Feel free to add more items here
config = {
    "beam_width" : 10,
    "lr"         : 2e-3,
    "epochs"     : 10,
    "batch_size" : 64  # Increase if your device can handle it
}

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [] # set of tranformations

### Data loaders

In [12]:
# get me RAMMM!!!!
import gc
gc.collect()

0

In [13]:
# Create objects for the dataset class
train_data = AudioDataset(root, "train-clean-100")
val_data = AudioDataset(root, "dev-clean")
test_data = AudioDatasetTest(root, "test-clean")

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset=train_data,
    num_workers=8,
    batch_size=config['batch_size'],
    pin_memory=True,
    shuffle=True,
    collate_fn=train_data.collate_fn
)

val_loader = torch.utils.data.DataLoader(
    dataset=val_data,
    num_workers=2,
    batch_size=config['batch_size'],
    pin_memory=True,
    shuffle=False,
    collate_fn=val_data.collate_fn
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_data,
    num_workers=2,
    batch_size=config['batch_size'],
    pin_memory=True,
    shuffle=False,
    collate_fn=test_data.collate_fn
)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [14]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([64, 1673, 28]) torch.Size([64, 214]) torch.Size([64]) torch.Size([64])


# NETWORK

## ASR Network

### Pyramid Bi-LSTM (pBLSTM)

In [15]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [16]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''

    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()
        
        self.blstm1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, bidirectional=True, batch_first=True)

    def forward(self, x_packed): # x_packed is a PackedSequence

        x, x_lens = nn.utils.rnn.pad_packed_sequence(x_packed, batch_first=True)
        x, x_lens = self.trunc_reshape(x, x_lens)

        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        # Pack Padded Sequence. What output(s) would you get?
        x_packed = nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)

        # Pass the sequence through bLSTM
        lstm_out, _ = self.blstm1(x_packed)

        return lstm_out

    def trunc_reshape(self, x, x_lens):
        # If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        if x.size(1) % 2 != 0:
            x_lens = x_lens - 1
            max_len = x_lens.max().item()
            x = x[:, :max_len, :]
        
        # Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        batch_size, time_steps, feature_dim = x.size()
        x = x.contiguous().view(batch_size, time_steps // 2, feature_dim * 2)
        
        # Reduce lengths by the same downsampling factor
        x_lens = x_lens // 2

        return x, x_lens

### Encoder

In [17]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size, emb_size=256):
        super(Encoder, self).__init__()


        self.embedding = nn.Conv1d(input_size, emb_size, kernel_size=3, padding=1)

        self.pBLSTMs = torch.nn.Sequential( # How many pBLSTMs are required?
            # Fill this up with pBLSTMs - What should the input_size be?
            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
            pBLSTM(input_size=emb_size * 2, hidden_size=emb_size),
            pBLSTM(input_size=emb_size * 4, hidden_size=encoder_hidden_size)
        )

    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        # Call the embedding layer
        x = x.transpose(1, 2)
        x = self.embedding(x)
        x = x.transpose(1, 2)
        
        # Pack Padded Sequence
        x_packed = nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        
        # Pass Sequence through the pyramidal Bi-LSTM layer
        encoder_out = self.pBLSTMs(x_packed)

        # Pad Packed Sequence
        encoder_out, encoder_lens = nn.utils.rnn.pad_packed_sequence(encoder_out, batch_first=True)

        return encoder_out, encoder_lens

### Decoder

In [18]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size=41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), torch.nn.BatchNorm1d(embed_size), PermuteBlock(),
            # Define your MLP arch. Refer HW1P2
            # Use Permute Block before and after BatchNorm1d() to match the size
            nn.Linear(embed_size, 512),
            nn.ReLU(),
            nn.Linear(512, output_size)
        )

        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        out = self.mlp(encoder_out)
        out = self.softmax(out)

        return out

In [19]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations  = torch.nn.Sequential(
            # Add Time Masking/ Frequency Masking
            # Hint: See how to use PermuteBlock() function defined above
            PermuteBlock(),
            tat.TimeMasking(time_mask_param=30),
            tat.FrequencyMasking(freq_mask_param=30),
            PermuteBlock(),
        )
        self.encoder = Encoder(input_size=input_size, encoder_hidden_size=embed_size//2)
        self.decoder = Decoder(embed_size=embed_size, output_size=output_size)
    
    def forward(self, x, lengths_x):

        if self.training:
            x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## Initialize ASR Network

In [20]:
model = ASRModel(
    input_size  = 28,
    embed_size  = 192,
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, x.to(device), lx)

ASRModel(
  (augmentations): Sequential(
    (0): PermuteBlock()
    (1): TimeMasking()
    (2): FrequencyMasking()
    (3): PermuteBlock()
  )
  (encoder): Encoder(
    (embedding): Conv1d(28, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (blstm1): LSTM(512, 256, batch_first=True, bidirectional=True)
      )
      (1): pBLSTM(
        (blstm1): LSTM(1024, 96, batch_first=True, bidirectional=True)
      )
    )
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): PermuteBlock()
      (1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PermuteBlock()
      (3): Linear(in_features=192, out_features=512, bias=True)
      (4): ReLU()
      (5): Linear(in_features=512, out_features=41, bias=True)
    )
    (softmax): LogSoftmax(dim=2)
  )
)
                                    Kernel Shape     Output Shape    Params  \
Layer                                                            

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_augmentations.PermuteBlock_0,-,"[64, 28, 1673]",NaN,NaN
1_augmentations.TimeMasking_1,-,"[64, 28, 1673]",NaN,NaN
2_augmentations.FrequencyMasking_2,-,"[64, 28, 1673]",NaN,NaN
3_augmentations.PermuteBlock_3,-,"[64, 1673, 28]",NaN,NaN
4_encoder.Conv1d_embedding,"[28, 256, 3]","[64, 256, 1673]",21760.0,35976192.0
5_encoder.pBLSTMs.0.LSTM_blstm1,-,"[41194, 512]",1576960.0,1572864.0
6_encoder.pBLSTMs.1.LSTM_blstm1,-,"[20584, 192]",861696.0,860160.0
7_decoder.mlp.PermuteBlock_0,-,"[64, 192, 418]",NaN,NaN
8_decoder.mlp.BatchNorm1d_1,[192],"[64, 192, 418]",384.0,192.0


# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [21]:
from torch.optim.lr_scheduler import StepLR

criterion = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=1e-5) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(labels=PHONEMES, beam_width=config['beam_width'], blank_id=0, log_probs_input=True)

scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

In [22]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

    # Look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    beam_results, beam_scores, timesteps, out_lens = decoder.decode(output, seq_lens=output_lens) #lengths - list of lengths

    pred_strings                    = []

    for i in range(output_lens.shape[0]):
        # Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        pred_indices = beam_results[i][0][:out_lens[i][0]].tolist()
        pred_string = ''.join([PHONEME_MAP[idx] for idx in pred_indices])
        pred_strings.append(pred_string)

    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

    for i in range(batch_size):
        # Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i]
        label_indices = label[i][:label_lens[i]].tolist()
        label_string = ''.join([PHONEME_MAP[idx] for idx in label_indices])
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size
    
    return dist

# Test Implementation

In [23]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    print(calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 734, 41])
torch.Size([734, 64, 41]) torch.Size([64, 265])
tensor(7.6756, device='cuda:0', grad_fn=<MeanBackward0>)
205.890625


# WandB

You will need to fetch your api key from wandb.ai

In [24]:
import wandb
wandb.login(key="930ceec05ed04b6998ffe53abfbc75f287963822")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shiwon1998. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/shiwon/.netrc


True

In [25]:
run = wandb.init(
    name = "early-submission", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

# Train Functions

In [26]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

## Training Setup

In [27]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [28]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
# epoch_model_path = # set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = '/home/shiwon/2024_CMU/IntroDL/HW3/P2/checkpoints/best_model.pth'

In [29]:
torch.cuda.empty_cache()
gc.collect()

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = optimizer.param_groups[0]['lr']

    train_loss = train_model(model, train_loader, criterion, optimizer)
    valid_loss, valid_dist = validate_model(model, val_loader, decoder, LABELS)
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    # save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    # wandb.save(epoch_model_path)
    # print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/10


Train:   0%|          | 0/446 [00:00<?, ?it/s]

wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


	Train Loss 2.2036	 Learning Rate 0.0020000
	Val Dist 25.8281%	 Val Loss 1.1604
Saved best model

Epoch: 2/10


	Train Loss 1.5954	 Learning Rate 0.0000200
	Val Dist 24.0317%	 Val Loss 1.0792
Saved best model

Epoch: 3/10


...

	Train Loss 1.5891	 Learning Rate 0.0000200
	Val Dist 22.6278%	 Val Loss 1.0150
Saved best model

Epoch: 8/10


	Train Loss 1.5837	 Learning Rate 0.0000200
	Val Dist 22.4707%	 Val Loss 1.0101
Saved best model

Epoch: 9/10


	Train Loss 1.5534	 Learning Rate 0.0000200
	Val Dist 22.0293%	 Val Loss 0.9899
Saved best model

Epoch: 10/10


	Train Loss 1.5630	 Learning Rate 0.0000200
	Val Dist 22.2295%	 Val Loss 1.0028


lr,█▁▁▁▁▁▁▁▁▁
train_loss,█▂▃▂▂▁▁▁▁▁
valid_dist,█▅▅▃▃▂▂▂▁▁
valid_loss,█▅▄▃▃▂▂▂▁▂
lr,2e-05
train_loss,1.56295
valid_dist,22.22946
valid_loss,1.0028


# Generate Predictions and Submit to Kaggle

In [55]:
# Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 20

test_decoder    = CTCBeamDecoder(labels=PHONEMES, beam_width=TEST_BEAM_WIDTH, blank_id=0)
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_string= decode_prediction(h, lh, test_decoder, PHONEME_MAP=LABELS)
    results.extend(prediction_string)

    del x, lx, h, lh
    torch.cuda.empty_cache()

Testing


100%|██████████| 41/41 [00:13<00:00,  2.94it/s]


In [59]:
data_dir = f"{root}/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

FileNotFoundError: [Errno 2] No such file or directory: '/home/shiwon/2024_CMU/IntroDL/HW3/P2/11785-f24-hw3p2/test-clean/random_submission.csv'

In [ ]:
!kaggle competitions submit -c hw3p2-785-f24 -f submission.csv -m "I made it!"
